In [4]:
import psycopg

connection_params = {
        "host": "localhost",
        "port": 5432,
        "dbname": "mobilitydb",
        "user": "postgres",
        "password": "postgres"
        }

In [5]:
# Define the base function template
base_function_template = """
CREATE OR REPLACE FUNCTION public.vulcan{index}(
            z integer, x integer, y integer, p_start text, p_end text)
RETURNS bytea
AS $$
    WITH bounds AS (
        SELECT ST_TileEnvelope(z,x,y) as geom
    ),
    trips_ AS (
        SELECT * FROM pymeos_demo LIMIT 100
    )
    ,
    vals AS (
        SELECT mmsi, numInstants(trip) as size, asMVTGeom(transform(attime(trip,span(p_start::timestamptz, p_end::timestamptz, true, true)),3857), transform((bounds.geom)::stbox,3857))
            as geom_times
        FROM (
            SELECT mmsi, trajectory::tgeompoint AS trip
            FROM trips_
        ) as ego, bounds
    ),
    mvtgeom AS (
        SELECT (geom_times).geom, (geom_times).times, size, mmsi
        FROM vals
    )
SELECT ST_AsMVT(mvtgeom) FROM mvtgeom
$$
LANGUAGE 'sql'
STABLE
PARALLEL SAFE;
"""

conn = psycopg.connect(**connection_params)
cursor = conn.cursor()

FRAMES_FOR_30_FPS = 12
total_frames = 1440
number_of_functions = total_frames // FRAMES_FOR_30_FPS 

# Generate 60 copies of the function
for i in range(number_of_functions):
    # Generate the function with the current index
    function_code = base_function_template.format(index=i*FRAMES_FOR_30_FPS)
    # Execute the function creation SQL
    cursor.execute(function_code)
    
# Commit the changes
conn.commit()

cursor.close()
conn.close()

In [1]:
1440/12

120.0

In [4]:
# Define the base function template
base_function_template = """
CREATE OR REPLACE FUNCTION public.zumba(
            z integer, x integer, y integer, p_start text, p_end text)
RETURNS bytea
AS $$
    WITH bounds AS (
        SELECT ST_TileEnvelope(z,x,y) as geom
    ),
    trips_ AS (
        SELECT * FROM pymeos_demo LIMIT 500
    )
    ,
    vals AS (
        SELECT mmsi, numInstants(trip) as size, asMVTGeom(transform(attime(trip,span(p_start::timestamptz, p_end::timestamptz, true, true)),3857), transform((bounds.geom)::stbox,3857))
            as geom_times
        FROM (
            SELECT mmsi, trajectory::tgeompoint AS trip
            FROM trips_
        ) as ego, bounds
    ),
    mvtgeom AS (
        SELECT (geom_times).geom, (geom_times).times, size, mmsi
        FROM vals
    )
SELECT ST_AsMVT(mvtgeom) FROM mvtgeom
$$
LANGUAGE 'sql'
STABLE
PARALLEL SAFE;
"""

conn = psycopg.connect(**connection_params)
cursor = conn.cursor()


function_code = base_function_template
# Execute the function creation SQL
cursor.execute(function_code)
    
# Commit the changes
conn.commit()

cursor.close()
conn.close()

In [ ]:
import psycopg2

# Define your connection parameters
connection_params = {
    'dbname': 'your_database',
    'user': 'your_username',
    'password': 'your_password',
    'host': 'your_host',
    'port': 'your_port'
}

# Number of functions to delete
FRAMES_FOR_30_FPS = 12
total_frames = 1440
number_of_functions = total_frames // FRAMES_FOR_30_FPS

# Connect to the database
conn = psycopg2.connect(**connection_params)
cursor = conn.cursor()

# Generate and execute DROP FUNCTION statements
for i in range(number_of_functions):
    function_name = f"public.vulcan{i*FRAMES_FOR_30_FPS}"
    drop_function_sql = f"DROP FUNCTION IF EXISTS {function_name}(integer, integer, integer, text, text);"
    cursor.execute(drop_function_sql)

# Commit the changes
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()
